After a few interactions with the author, it now install properly. We do need to find a better way to parallelize it. But since Tri is done with the new pre-processing pipeline, let's properly link the BAM files first. we'll need to generate new ones for the simplex only samples, so let's see what we have so far:

In [2]:
%%bash

cd ~/data/cnv/mdts
cp ../conifer84/samples.txt .

echo "Samples to re-run:"
while read s; do
    if [ ! -e /data/NCR_SBRB/ADHDQTL/GATK/BAM/${s}/${s}_trimmed_sorted_RG_markduplicate_recalibrated.bam ]; then
        echo $s;
    fi;
done < samples.txt

Samples to re-run:
CLIA_400185
CLIA_400200
CLIA_400193
CLIA_400195
CLIA_400148
CLIA_400188
CLIA_400189
CLIA_400191
CLIA_400181
CLIA_400203
CLIA_400202
CLIA_400201
CLIA_400192
CLIA_400184
CLIA_400206
CLIA_400186
CLIA_400204
CLIA_400125
CLIA_400190
CLIA_400159
CLIA_400167
CLIA_400179
CLIA_400197
CLIA_400215
CLIA_400171
CLIA_400170
CLIA_400173
CLIA_400172
CLIA_400175
CLIA_400174
CLIA_400177
CLIA_400176
CLIA_400157
CLIA_400156
CLIA_400154
CLIA_400153
CLIA_400152
CLIA_400151
CLIA_400149
CLIA_400135
CLIA_400134
CLIA_400137
CLIA_400136
CLIA_400131
CLIA_400130
CLIA_400133
CLIA_400132
CLIA_400180
CLIA_400187
CLIA_400158
CLIA_400139
CLIA_400138
CLIA_400122
CLIA_400199
CLIA_400198
CLIA_400178
CLIA_400210
CLIA_400211
CLIA_400212
CLIA_400213
CLIA_400214
CLIA_400196
CLIA_400216
CLIA_400194
CLIA_400147
CLIA_400205
CLIA_400121
CLIA_400182
CLIA_400127
CLIA_400166
CLIA_400183
CLIA_400128
CLIA_400129
CLIA_400162
CLIA_400163
CLIA_400160
CLIA_400161
CLIA_400144
CLIA_400123
CLIA_400150
CLIA_400140
CLIA_4001

Well, that's disheartening... do we have anyone ready?

In [3]:
%%bash

cd ~/data/cnv/mdts

echo "Samples ready:"
while read s; do
    if [ -e /data/NCR_SBRB/ADHDQTL/GATK/BAM/${s}/${s}_trimmed_sorted_RG_markduplicate_recalibrated.bam ]; then
        echo $s;
    fi;
done < samples.txt

Samples ready:


Alright then... we'll need to re-run everyone.

Since we're re-running this, it'll be interesting to try different BAM pipelines:

1. Standard (BWA MEM + Picard + Base recalibration)
2. No base recalibration (BWA MEM + Picard)
3. mrsFast (as suggested in CONIFER)

Running mrsFast might not be worth it, and it's somewhat older software (2014). So, if we end up doing it, we might as well run some of their own CNV tools (http://mrcanavar.sourceforge.net/manual.html)

Also, note that I'm running the WPS version of the trimming, because I first tried the CCGO/CLIA version, and the bad KMER contents were still there. I'm pretty sure it's because the last wave had CLIA samples with WPS, and the issue is with the wave itself, not the sample code. But I'll double check the QC files when they're done running before going ahead with all different CNV pipelines. There's a chance we'll be removing too much for some samples (i.e. the CLIA samples not processed in the last wave, but who are part of the simplex study). But hopefully it won't be too bad.

In [ ]:
%%bash

cd /data/NCR_SBRB/simplex

while read s; do
    echo "bash ~/autodenovo/gatk_upToSingleCalls.sh $s" >> swarm.single;
done < ~/data/cnv/mdts/samples.txt;

[sudregp@cn2350 simplex]$ head swarm.single
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400185
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400200
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400193
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400195
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400148
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400188
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400189
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400191
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400181
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400203

[sudregp@cn2350 simplex]$ wc -l swarm.single
84 swarm.single
[sudregp@cn2350 simplex]$ swarm -f swarm.single -t 32 -g 120 --job-name gatk_single --logdir trash --time=48:00:00 --gres=lscratch:1
00
63296498

In [ ]:
%%R

library(MDTS, lib='~/R')
library(BSgenome.Hsapiens.UCSC.hg19)
genome = BSgenome.Hsapiens.UCSC.hg19
map_file = "chr1.map.bw"
setwd('~/data/cnv/mdts')
pD = getMetaData("simplex84.ped")
bins = calcBins(pD, n=5, readLength=100, medianCoverage=150,
                minimumCoverage=5, genome=genome, mappabilityFile=map_file)

# it might be possible to make this go faster if we split the genome by chromosomes?

counts = calcCounts(pD, bins, rl=100)
mCounts <- normalizeCounts(counts, bins)
md <- calcMD(mCounts, pD)
cbs <- segmentMD(md=md, bins=bins)
denovo <- denovoDeletions(cbs, mCounts, bins)

NEED TO DOWNLOAD MAPABILITY FILES FOR ALL CHROMOSOMES. SOMETHING LIKE THIS: http://hgdownload.soe.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeMapability/. BUT SHOULD HECK THE QC FILES FOR A BETTER READ LENGTH ESTIMATE.